In [1]:
import os

os.getcwd()
if '.git' not in os.listdir(): 
    os.chdir('./../../../../')
os.listdir()


['run.sh',
 'requirements.txt',
 'ml',
 '.idea',
 'bot_discord',
 'venv',
 '.git',
 'log',
 '.gitignore']

In [2]:
import datetime


import fasttext
import keras_core as keras
import numpy as np
import tensorflow as tf
from keras_core.models import load_model
from sklearn.model_selection import train_test_split
from tensorboard.plugins.hparams import api as hp


Using TensorFlow backend


2024-07-02 10:13:53.309357: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 10:13:53.309407: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 10:13:53.309442: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-02 10:13:53.320736: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## import fastTest model

In [3]:
#

## Configurations

In [4]:
params_grid = {
    "model": "mlp",
    "embedder": "fasttext",
    "embedding_data": "cc.fr.300.bin",
    "dataset": "fr_hf.csv",
    "batch_size": 300
}

In [5]:
data_folder_src = 'ml/ml_core/data/processed/'
ml_core_folder_path = 'ml/ml_core/'
data_path_src = f'{data_folder_src}{params_grid["dataset"]}'
date_str = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
experiment_name = f'deepwoke_{params_grid["embedder"]}_{params_grid["model"]}_{params_grid["dataset"]}_{date_str}'

log_dir = f"{ml_core_folder_path}log/fit/{experiment_name}"
model_weight_dst = f'{ml_core_folder_path}model_weights/{experiment_name}-model.keras'

In [6]:

vector_path = f'{ml_core_folder_path}embedded_vector/{params_grid["embedding_data"]}_{params_grid["dataset"]}'
vector_x_dst = f'{vector_path}.x.npy'
vector_y_dst = f'{vector_path}.y.npy'

# loading dataset

In [7]:
X = np.load(vector_x_dst)
y = np.load(vector_y_dst)

## text to vector

## building and training the model

### callbacks

In [8]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
    monitor='loss'
)


In [9]:

class EarlyStoppingLogging(tf.keras.callbacks.Callback):
    def __init__(self, early_stopping_callback, log_dir):
        super().__init__()
        self.early_stopping = early_stopping_callback
        self.stopped_epoch = 0
        self.writer = tf.summary.create_file_writer(log_dir)

    def on_epoch_end(self, epoch, logs=None):
        if self.early_stopping.stopped_epoch > 0:
            self.stopped_epoch = self.early_stopping.stopped_epoch
            with self.writer.as_default():
                tf.summary.scalar('early_stopping_epoch', self.stopped_epoch, step=epoch)
                self.writer.flush()


In [10]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_logging_callback = EarlyStoppingLogging(early_stopping, log_dir)

2024-07-02 10:13:58.544902: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 10:13:58.592938: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 10:13:58.592979: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 10:13:58.595193: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 10:13:58.595247: I tensorflow/compile

In [11]:
callback = [
    early_stopping,
    tensorboard_callback,
    hp.KerasCallback(log_dir, params_grid),
    early_stopping_logging_callback,
]

### training the model

In [12]:

from ml.ml_core.src.model.model_factory import model_factory

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = model_factory(params_grid['model'], 300, 1)
print(model.summary)

model = keras.models.Sequential()
model.add(keras.layers.Dense(128, input_dim=300, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(34, activation='relu'))
model.add(keras.layers.Dense(1, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary)

model.fit(X_train,
          y_train,
          epochs=500,
          batch_size=params_grid['batch_size'],
          callbacks=callback,
          validation_data=(X_test, y_test)
          )

<bound method Model.summary of <Functional name=functional_1, built=True>>
<bound method Model.summary of <Sequential name=sequential, built=True>>


/home/arys/miniconda3/envs/DEV_IA/lib/python3.10/site-packages/keras_core/src/layers/core/dense.py:73: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/500


2024-07-02 10:14:01.189646: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x910d520 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-02 10:14:01.189687: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-07-02 10:14:01.215919: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-02 10:14:01.353301: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700


36/49 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2409 - loss: 412.2628

2024-07-02 10:14:02.040383: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.2405 - loss: 411.2651 - val_accuracy: 0.2313 - val_loss: 387.1880
Epoch 2/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2426 - loss: 414.8920 - val_accuracy: 0.2313 - val_loss: 387.1880
Epoch 3/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2382 - loss: 407.4946 - val_accuracy: 0.2313 - val_loss: 387.1880
Epoch 4/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2404 - loss: 410.9908 - val_accuracy: 0.2313 - val_loss: 387.1880
Epoch 5/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2414 - loss: 412.8482 - val_accuracy: 0.2313 - val_loss: 387.1880
Epoch 6/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2402 - loss: 410.5568 - val_accuracy: 0.2313 - val_loss: 387.1880
Epoch 7/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2373 - loss: 405.8844 - val_accuracy: 0.2313 - val_loss: 387.1880
Epoch 8/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2316 - loss: 395.9778 - val_a

# save model

In [13]:
model.save(model_weight_dst)

### testing with custom sentence

In [14]:
#model_weight_dst = f'../../model_weights/fasttext_nlp_20240613-131608-model.keras'
new_model = load_model(model_weight_dst)

In [15]:
ft = fasttext.load_model('ml/ml_core/data/embedding_data/cc.fr.300.bin')


from ml.ml_core.src.embedding.embedding_utils import text_to_vector

text_nice = [
    "la bonjour je suis un type sympas",
    "bonjour ça va les gens",
    "j'aime pas trop ce mec",
    "cette meuf est cool",
    "les meuf sont trop cool",
    "les mecs sont des gens stylé",
    "le mec est stylé",
    "nan mais mon manager exagère quand il dit que je suis pas assez productif et que je travail pas assez, franchement il abuse",
    "les hommes cis sont toxiques pour la société car ils imposent leur vision paternaliste",
    "Les hommes sont moins matures que les femmes quand il s'agit de prendre des décisions"

]

text_no_nice = [
    "regarde moi cette connasse",
    "les femme leur place c'est à la cuisine",
    "femme au volant, accident",
    "les femmes on peux pas leur faire confiance",
    "Pour une transphobe son passing est pas ouf",
    "Elle l’ouvre quand même beaucoup celle là pour quelqu’un qu’est pas capable d’éduquer ses enfants",
    "Dis donc, c’est pas parce que t’es mal baisée que tu peux tout te permettre ????",
    "Moi ce que j’aime bien chez les filles comme toi c’est que t’es pas du genre prise de tête",
    "les femmes c'est comme le fromage, quand elles puent c'est qu'elles sont bonnes",
    "Oh les grosses tepus",
    "Faut pas t'étonner si tu te fais violer vu comment t'es habillée",
    "Tu deverais avoir des enfants maintenant avant que ce soit trop tard",
    "Vous seriez une collaboratrice idéal pour notre entreprise, malheureusement nous cherchons quelqu’un disponible sur le long terme, et nous craignons qu’à votre âge des changements familiaux ne vous permettent plus de vous investir autant que besoins"
]
wordsvec_nice = [np.expand_dims(text_to_vector(text, ft), axis=0) for text in text_nice]
wordsvec_no_nice = [np.expand_dims(text_to_vector(text, ft), axis=0) for text in text_no_nice]

print("nice output")
for i in range(0, len(wordsvec_nice)):
    prediction = new_model.predict(wordsvec_nice[i], verbose=False)
    print(f"{text_nice[i]} : {prediction > 0.5}")
    print(f"{prediction}")

print("\n\nnot nice output")
for i in range(0, len(wordsvec_no_nice)):
    prediction = new_model.predict(wordsvec_no_nice[i], verbose=False)
    print(f"{text_no_nice[i]} : {prediction > 0.5}")
    print(f"{prediction}")

nice output
la bonjour je suis un type sympas : [[ True]]
[[1.]]
bonjour ça va les gens : [[ True]]
[[1.]]
j'aime pas trop ce mec : [[ True]]
[[1.]]
cette meuf est cool : [[ True]]
[[1.]]
les meuf sont trop cool : [[ True]]
[[1.]]
les mecs sont des gens stylé : [[ True]]
[[1.]]
le mec est stylé : [[ True]]
[[1.]]
nan mais mon manager exagère quand il dit que je suis pas assez productif et que je travail pas assez, franchement il abuse : [[ True]]
[[1.]]
les hommes cis sont toxiques pour la société car ils imposent leur vision paternaliste : [[ True]]
[[1.]]
Les hommes sont moins matures que les femmes quand il s'agit de prendre des décisions : [[ True]]
[[1.]]


not nice output
regarde moi cette connasse : [[ True]]
[[1.]]
les femme leur place c'est à la cuisine : [[ True]]
[[1.]]
femme au volant, accident : [[ True]]
[[1.]]
les femmes on peux pas leur faire confiance : [[ True]]
[[1.]]
Pour une transphobe son passing est pas ouf : [[ True]]
[[1.]]
Elle l’ouvre quand même beaucoup cell